In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from PIL import Image, ImageTk
import os
import geopandas as gpd

# Define the function to run the GUI
def run_gui():
    # Clear input variables
    job_id = None
    pfi = None
    analyst = None
    ndwi_threshold = None
    colour_map = None 
    base_dir = None
    aoi = None
    gui_path = None

    # Option lists
    name_options = ['Jason Dechastel', 'Lasinidu Jayarathna', 'Eva Kovaks', 'Craig Turner']

    # Create corporate colours
    Cybernetic = "#275c6d"  # Primary
    Prosperity = "#077297"  # Secondary
    Bold_Blue = "#023a57"   # Secondary

    Mangrove = "#4b623b"    # Primary
    Waterhole = "#a69b5e"   # Secondary
    H20 = "#315450"         # Secondary

    Terrain = "#8c4d36"     # Primary
    Ochre = "#ac6d29"       # Secondary
    Sunrise = "#c48c33"     # Secondary

    # Assign colours
    main_bg_colour =  Prosperity 
    frame_bg_colour = Cybernetic
    label_bg_colour = Cybernetic
    entry_bg_colour = "black" # Black entry widget background
    button_bg_colour = "white"  # White button

    # Functions to handle file selection
    def select_file(entry_widget):
        filename = filedialog.askopenfilename()
        entry_widget.delete(0, tk.END)
        entry_widget.insert(0, filename)
        check_shapefile_type(filename)  # Check shapefile type after selection

    def select_directory(entry_widget):
        directory = filedialog.askdirectory()
        entry_widget.delete(0, tk.END)
        entry_widget.insert(0, directory)

    # Function to check shapefile type
    def check_shapefile_type(shapefile):
        try:
            # Load the shapefile
            gdf = gpd.read_file(shapefile)
            # Check if it's a point or polygon
            if gdf.geometry.iloc[0].geom_type == 'Point':
                show_buffer_distance()
            else:
                hide_buffer_distance()
        except Exception as e:
            print(f"Error loading shapefile: {e}")
            hide_buffer_distance()

    # Show buffer distance input if the shapefile is a point
    def show_buffer_distance():
        buffer_label.grid(row=5, column=0, sticky="e")
        buffer_entry.grid(row=5, column=1, padx=5, pady=5, sticky="w")

    # Hide buffer distance input if the shapefile is not a point
    def hide_buffer_distance():
        buffer_label.grid_forget()
        buffer_entry.grid_forget()

    def submit_form():
        # Retrieve input values
        job_id = field1_entry.get()
        name = name_combobox.get()
        shapefile = source2_entry.get()

        # Check if buffer_distance is a valid number, and convert to float if it is
        buffer_distance = None
        if buffer_entry.winfo_ismapped():  # Check if buffer distance field is visible
            try:
                buffer_distance = float(buffer_entry.get())  # Convert to float
                if buffer_distance <= 0:
                    raise ValueError("Buffer distance must be positive.")
            except ValueError as e:
                print(f"Error: Invalid buffer distance. {e}")
                return  # Exit if invalid buffer distance

        output_folder = output1_entry.get()

        # Save input data to a file so the main script can access it
        data = {
            "job_id": job_id,
            "name": name,
            "shapefile": shapefile,
            "buffer_distance": buffer_distance,
            "output_folder": output_folder,
        }

        with open("gui_output.json", "w") as f:
            import json
            json.dump(data, f)

        print("Form submitted! Data saved to gui_output.json")
        root.quit()  # Close the GUI after submission

    # Main window creation and setup
    root = tk.Tk()
    root.title("Storage Crest Polygon Tool")
    root.configure(bg=main_bg_colour)
    root.attributes('-topmost', True)
                    
    # Load an image using PIL's Image
    image_path = 'Qld-CoA-Stylised-1L-mono_rev.png'
    if os.path.exists(image_path):
        image = Image.open(image_path)
        photo = ImageTk.PhotoImage(image)
    else:
        photo = None

    # Create a frame to hold the image and the title
    header_frame = tk.Frame(root, bg=frame_bg_colour)
    header_frame.grid(row=0, column=0, columnspan=3, sticky="nw", padx=5, pady=5)

    # Display image if found
    if photo:
        image_label = tk.Label(header_frame, image=photo, bg=frame_bg_colour)
        image_label.grid(row=0, column=0, sticky="nw")

    # Display title
    title_label = tk.Label(header_frame, text="Storage Crest Polygon Tool\n \nStrategic Compliance, Intelligence, & Investigations ", 
                           font=("Arial", 22), fg="#ffffff", bg=label_bg_colour)
    title_label.grid(row=0, column=1, sticky="w")

    # Create labels and entry widgets for each field
    tk.Label(root, text="Job ID:").grid(row=1, column=0, sticky="e")
    field1_entry = tk.Entry(root, width=28)
    field1_entry.grid(row=1, column=1, padx=5, pady=5, sticky="w")

    ttk.Label(root, text="Name:").grid(row=3, column=0, sticky="e")
    name_combobox = ttk.Combobox(root, values=name_options, width=25)
    name_combobox.grid(row=3, column=1, padx=5, pady=5, sticky="w")
    name_combobox.set('Craig Turner')

    tk.Label(root, text="Shapefile:").grid(row=4, column=0, sticky="e")
    source2_entry = tk.Entry(root, width=40)
    source2_entry.grid(row=4, column=1, padx=5, pady=5, sticky="we")
    tk.Button(root, text="Browse", command=lambda: select_file(source2_entry)).grid(row=4, column=2, padx=5, pady=5, sticky="w")
    
    # Initially hide buffer distance
    buffer_label = tk.Label(root, text="Buffer Distance:")
    buffer_entry = tk.Entry(root, width=28)
        
    tk.Label(root, text="Output Folder:").grid(row=6, column=0, sticky="e")
    output1_entry = tk.Entry(root, width=40)
    output1_entry.grid(row=6, column=1, padx=5, pady=5, sticky="we")
    tk.Button(root, text="Browse", command=lambda: select_directory(output1_entry)).grid(row=6, column=2, padx=5, pady=5, sticky="w")

    # Submit button
    submit_button = tk.Button(root, text="Submit", command=submit_form)
    submit_button.grid(row=7, column=0, columnspan=3, pady=10)

    # Run the GUI
    root.mainloop()

# Run the GUI only if this script is executed directly
if __name__ == "__main__":
    run_gui()
